In [470]:
import pandas as pd
import numpy as np

In [471]:
# create dataframes from csv files

schools_df = pd.read_csv('resources/schools_complete.csv')
students_df = pd.read_csv('resources/students_complete.csv')

# merge into one dataframe 
merged_df = pd.merge(schools_df, students_df, on="school_name")


In [472]:
merged_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [473]:
# rename columns
merged_df = merged_df.rename(columns={"school_name": "School Name", 
                                    "type": "Type", "size": "Size", 
                                    "budget": "Budget", "student_name": "Student Name", "gender": "Gender",
                                    "grade": "Grade", "reading_score": "Reading Score", "math_score": "Math Score"})
merged_df.head()

,School ID,School Name,Type,Size,Budget,Student ID,Student Name,Gender,Grade,Reading Score,Math Score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [474]:
# get unique school count
school_count = len(merged_df["School Name"].unique())
school_count

15

In [475]:
# get count of students
student_count = len(merged_df)
student_count

39170

In [476]:
# calculate total budget
total_budget = merged_df[["School Name", "Budget"]]
total_budget = total_budget.drop_duplicates()
total_budget = total_budget["Budget"].sum()
total_budget

24649428

In [477]:
# calculate average (mean) math score
average_math = merged_df["Math Score"].mean()
average_math

78.98537145774827

In [478]:
#calculate average (mean) reading score
average_reading = merged_df["Reading Score"].mean()
average_reading

81.87784018381414

In [479]:
# percentage passing math
passing_math_count = merged_df[(merged_df["Math Score"] >= 70)].count()["Student Name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [480]:
# percentage passing reading
passing_reading_count = merged_df[(merged_df["Reading Score"] >= 70)].count()["Student Name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [481]:
# calculate overall passing percentage
passing_math_reading_count = merged_df[
    (merged_df["Math Score"] >= 70) & (merged_df["Reading Score"] >= 70)
].count()["Student Name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [482]:
# create district summary 
district_summary = pd.DataFrame({"Total Schools": [school_count],
                              "Total Students": student_count,
                                 "Total Budget": total_budget,
                              "Average Math Score": average_math,
                              "Average Reading Score": average_reading,
                                "% Passing Math": passing_math_percentage,
                                "% Passing Reading": passing_reading_percentage,
                                "Overall Passing %": overall_passing_rate})
# format columns
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [483]:
# select all school types
school_types = merged_df["Type"].unique()
school_types

array(['District', 'Charter'], dtype=object)

In [484]:
# per school type student counts
charter_count = merged_df[merged_df["Type"] == "Charter"].count()["Student Name"]
district_count = merged_df[merged_df["Type"] == "District"].count()["Student Name"]

charter_count

12194

In [485]:
district_count

26976

In [486]:
# per school budget
per_school_budget = schools_df.set_index('school_name')['budget']


In [487]:
# group by school 
group_by_school = merged_df.set_index("School Name").groupby(["School Name"])

In [488]:
# group by school type
school_types = schools_df.set_index('school_name')['type']


In [489]:
# students per school
students_by_school =  group_by_school["Student ID"].count()

In [490]:
# budget per student
budget_per_student = schools_df.set_index('school_name')['budget'] / schools_df.set_index('school_name')['size']

In [491]:
# average scores by school 
average_math = group_by_school['Math Score'].mean()
average_reading = group_by_school['Reading Score'].mean()

In [492]:
# calculate percentage passing
passing_math = merged_df[merged_df['Math Score'] >= 70].groupby('School Name')['Student ID'].count() / students_by_school 
passing_math = passing_math * 100
passing_reading = merged_df[merged_df['Reading Score'] >= 70].groupby('School Name')['Student ID'].count() / students_by_school 
passing_reading = passing_reading * 100
overall_passing = merged_df[(merged_df['Reading Score'] >= 70) & (merged_df['Math Score'] >= 70)].groupby('School Name')['Student ID'].count()/students_by_school
overall_passing = overall_passing * 100

In [502]:
# create school summary 
school_summary = pd.DataFrame({"School Type": school_types,
                            "Total Students": students_by_school,
                            "Per Student Budget": budget_per_student,
                            "Total School Budget": per_school_budget,
                            "Average Math Score": average_math,
                            "Average Reading Score": average_reading,
                            "% Passing Math": passing_math,
                            "% Passing Reading": passing_reading,
                            "Overall Passing %": overall_passing
})

# format columns
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)

school_summary


,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
Bailey High School,District,"4,976",$628.00,"$3,124,928.00",77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858",$582.00,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949",$639.00,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739",$644.00,"$1,763,916.00",77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468",$625.00,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635",$652.00,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,$581.00,"$248,087.00",83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917",$655.00,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761",$650.00,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,$609.00,"$585,858.00",83.839917,84.044699,94.594595,95.945946,90.540541


In [503]:
# sort highest performing schools by % overall passing
top_schools = school_summary.sort_values("Overall Passing %", ascending = False)
top_schools.head(5)

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
Cabrera High School,Charter,"1,858",$582.00,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635",$638.00,"$1,043,130.00",83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468",$625.00,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283",$578.00,"$1,319,574.00",83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,$609.00,"$585,858.00",83.839917,84.044699,94.594595,95.945946,90.540541


In [504]:
# sort bottom performing schools by overall passing %
bottom_schools = school_summary.sort_values("Overall Passing %")
bottom_schools.head(5)

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
Rodriguez High School,District,"3,999",$637.00,"$2,547,363.00",76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949",$639.00,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917",$655.00,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635",$652.00,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761",$650.00,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172


In [505]:
# calculate average math scores by grade 
ninth_math = students_df.loc[students_df['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = students_df.loc[students_df['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = students_df.loc[students_df['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = students_df.loc[students_df['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})

math_scores.index.name = "School Name"

math_scores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [506]:
# calculate average reading scores by grade 
ninth_reading = students_df.loc[students_df['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_reading = students_df.loc[students_df['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_reading = students_df.loc[students_df['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_reading = students_df.loc[students_df['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})

reading_scores.index.name = "School Name"

reading_scores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [507]:
# create bins and labels for spending
spending_bins = [0, 584.999, 614.999, 644.999, 999999]
labels = ["< $585", "$585-630", "$630-645", "$645-680"]

# make a copy of the school_summary dataframe
school_spending = school_summary.copy()

# add bins 
school_spending["Spending Ranges (Per Student)"] = pd.cut(budget_per_student, spending_bins, labels=labels)

school_spending


,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %,Spending Ranges (Per Student)
Bailey High School,District,"4,976",$628.00,"$3,124,928.00",77.048432,81.033963,66.680064,81.933280,54.642283,$630-645
Cabrera High School,Charter,"1,858",$582.00,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769,< $585
Figueroa High School,District,"2,949",$639.00,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,"2,739",$644.00,"$1,763,916.00",77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,"1,468",$625.00,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455,$630-645
Hernandez High School,District,"4,635",$652.00,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,$581.00,"$248,087.00",83.803279,83.814988,92.505855,96.252927,89.227166,< $585
Huang High School,District,"2,917",$655.00,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,"4,761",$650.00,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,$609.00,"$585,858.00",83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [508]:
# calculate averages
spending_math_scores = school_spending.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending.groupby(["Spending Ranges (Per Student)"])["Overall Passing %"].mean()


# create school spending summary dataframe
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    '% Passing Math': spending_passing_math,
    '% Passing Reading': spending_passing_reading,
    "Overall Passing %": overall_passing_spending
            
})

# add formatting
spending_summary["Average Math Score"] = spending_summary["Average Math Score"].map("{:.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:.2f}".format)
spending_summary["% Passing Math"] = spending_summary["% Passing Math"].map("{:.2f}%".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:.2f}%".format)
spending_summary["Overall Passing %"] = spending_summary["Overall Passing %"].map("{:.2f}%".format)

spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
Spending Ranges (Per Student),,,,,
< $585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,83.60,83.89,94.23%,95.90%,90.22%
$630-645,79.08,81.89,75.67%,86.11%,66.11%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


In [509]:
# summarize scores by school size

#. create bins and labels for school size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary["School Size"] = pd.cut(students_by_school, size_bins, labels=labels)
school_summary

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %,School Size
Bailey High School,District,"4,976",$628.00,"$3,124,928.00",77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,"1,858",$582.00,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,"2,949",$639.00,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,"2,739",$644.00,"$1,763,916.00",77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,"1,468",$625.00,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,"4,635",$652.00,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,$581.00,"$248,087.00",83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,"2,917",$655.00,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,"4,761",$650.00,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,$609.00,"$585,858.00",83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [510]:
# calculate scores by size of school
math_scores_by_size = school_summary.groupby(["School Size"])["Average Math Score"].mean()
reading_scores_by_size = school_summary.groupby(["School Size"])["Average Reading Score"].mean()
passing_math_by_size = school_summary.groupby(["School Size"])["% Passing Math"].mean()
passing_reading_by_size = school_summary.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_by_size = school_summary.groupby(["School Size"])["Overall Passing %"].mean()

# create school size summary dataframe
school_size_summary = pd.DataFrame(
    {
        "Average Math Score": math_scores_by_size,
        "Average Reading Score": reading_scores_by_size,
        "% Passing Math": passing_math_by_size,
        "% Passing Reading": passing_reading_by_size,
        "Overall Passing %": overall_passing_by_size,
    }
)

# add formatting
school_size_summary["Average Math Score"] = school_size_summary["Average Math Score"].map("{:.2f}".format)
school_size_summary["Average Reading Score"] = school_size_summary["Average Reading Score"].map("{:.2f}".format)
school_size_summary["% Passing Math"] = school_size_summary["% Passing Math"].map("{:.2f}%".format)
school_size_summary["% Passing Reading"] = school_size_summary["% Passing Reading"].map("{:.2f}%".format)
school_size_summary["Overall Passing %"] = school_size_summary["Overall Passing %"].map("{:.2f}%".format)

school_size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


In [511]:
# group scores by school type
average_math_by_type = school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_by_type = school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_passing_math_by_type = school_summary.groupby(["School Type"])["% Passing Math"].mean() 
average_passing_reading_by_type = school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_overall_passing_by_type = school_summary.groupby(["School Type"])["Overall Passing %"].mean()

# create dataframe
school_type_summary = pd.DataFrame(
    {
        "Average Math Score": average_math_by_type,
        "Average Reading Score": average_reading_by_type,
        "% Passing Math": average_passing_math_by_type,
        "% Passing Reading": average_passing_reading_by_type,
        "Overall Passing %": average_overall_passing_by_type,
    }
)

# add formatting
school_type_summary["Average Math Score"] = school_type_summary["Average Math Score"].map("{:.2f}".format)
school_type_summary["Average Reading Score"] = school_type_summary["Average Reading Score"].map("{:.2f}".format)
school_type_summary["% Passing Math"] = school_type_summary["% Passing Math"].map("{:.2f}%".format)
school_type_summary["% Passing Reading"] = school_type_summary["% Passing Reading"].map("{:.2f}%".format)
school_type_summary["Overall Passing %"] = school_type_summary["Overall Passing %"].map("{:.2f}%".format)

# display results
school_type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
